# Assignment 1, 8-Puzzle Programming

Your homework must be implemented in this Notebook file. 
You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
You need to implement the three (four for grads) searching functions and the print result functions.
For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.


In [1]:
from numpy import *
import numpy as np
import queue 
import gc
import time
import copy
import io
import sys   
sys.setrecursionlimit(1000000)
import itertools
from collections import deque,namedtuple
from queue import PriorityQueue
from hashlib import sha1

GOAL_STATE = [ 0,1, 2, 3, 4, 5, 6, 7, 8]
print_state=list()
GOALIND = [(0, (0, 0)),(8, (2, 2)),  (1, (0, 1)), (2, (0, 2)), (3, (1, 0)), (4, (1, 1)), (6, (1, 2)), (6, (2, 0)), (7, (2, 1))]
Coord = namedtuple("Coord", "y x")

In [2]:
#implementation of function "Iterative_deepening_DFS" and DFS
#Ziqin Yan
result=list()
class Node:
    
    def __init__(self, data, g = 0 ):
        self.data = np.array(data, dtype=np.int8) 
        self.g = g
        #self.tiles.flags.writeable = False 
        self.parent=list()
        

    def find_tile(self, tile_id):
        return Coord._make(
          np.concatenate(np.where(self.data == tile_id)))
    def hash_titles(self):
        #compare faster
        return sha1(self.data).hexdigest()

    def coord_valid(self, coord):
        return (coord.y >= 0 and coord.x >= 0 and
                coord.y < self.data.shape[0] and coord.x < self.data.shape[1])

  
    def swap(self, coord):
        old_empty = self.find_tile(0)
        swapped_data = np.copy(self.data)
    
        # Replace 
        swapped_data[old_empty.y][old_empty.x] = self.data[coord.y][coord.x]
        
        swapped_data[coord.y][coord.x] = 0
        
        
        new_node=Node(swapped_data, self.g + 1)
        new_node.parent.extend(self.parent)
        new_node.parent.append(self.data)
        
        return new_node
        
    def find_neighbors(self, coord):
        def delta_to_coord(delta):
            return Coord(coord.y + delta[0], coord.x + delta[1]) 
    
        return set(filter(self.coord_valid,
          map(delta_to_coord, [(0,1), (1,0), (0,-1), (-1,0)])))

    def move(self):
        return self.find_neighbors(self.find_tile(0))

    def children(self):
        
        return map(self.swap, self.move())




def swap(state,index1,index2):
    temp=state[index1]
    state[index1]=state[index2]
    state[index2]=temp
    return state

def move_state(state,move):
    next_state=list(state)
    index=state.index(0)
   
    if move =='R':
        if index % 3 == 2:
            return None
        next_state = swap(next_state,index,index+1)
    elif move=='L':
        if index%3 ==0:
            return None
        next_state=swap(next_state,index,index-1)
    elif move == 'U':
        if index<3:
            return None
        next_state=swap(next_state,index,index-3)
    elif move == 'D':
        if index > 5:
            return None
        next_state=swap(next_state,index,index + 3)
    else:
        return None
    return next_state

def move(state):
    next_state=[]
    ty_state = move_state(state,'R')
    if ty_state is not None:
        next_state.append(ty_state)
        
    ty_state =move_state(state,'L')
    if ty_state is not None:
        next_state.append(ty_state)
        
    ty_state =move_state(state,'U')
    if ty_state is not None:
        next_state.append(ty_state)
        
    ty_state =move_state(state,'D')
    if ty_state is not None:
        next_state.append(ty_state)

    
    
    #print("ceng",next_state)
    return next_state 


    
            
def dfs_search(list_input):
   
    
    puzzle=list()
    puzzle_cp=set()
    puzzle_cp.add(tuple(list_input))
    
    puzzle.append(list_input)
    explored = set()
    print_state=list()
    count=0
    #puzzle_cp.add(list_input)
    t=0
    while puzzle:
        count+=1
        #print("puzzle:", puzzle)
        c_state = puzzle.pop()
        print_state.append(c_state)
        
        puzzle_cp.remove(tuple(c_state))
        explored.add(tuple(c_state))
        if c_state == GOAL_STATE:
            step=len(explored)
            del puzzle_cp
            print(count)
            
            return c_state , print_state,step,count
        # reversed because avoid BUG
        #some reason:http://www.cnblogs.com/bananaplan/p/remove-listitem-while-iterating.html
        
        for move_state in reversed(move(c_state)):
            
            if  tuple(move_state) not in explored and tuple(move_state) not in puzzle_cp :
                #dfs_search(move_state)
                #print(move_state)
                puzzle.append(move_state)
                puzzle_cp.add(tuple(move_state))
    return None
def iterativeDeepeningDepthFirst(root_node):
    goal =np.arange(9).reshape(3,3)
    iterations = 0 
    puzzle=list()
   
    for depth in itertools.count(): 
        puzzle=list()
        queue = [root_node]
        puzzle.append(root_node)
    
        visited = {} 
        #print("depth",depth)
        while len(queue) > 0:
            iterations = iterations + 1
        
            #puzzle.pop
            node = queue.pop() 
        
            visited[node.hash_titles()] = node.g 
            
            if  np.all(node.data==goal):
                
                node.parent.append(node.data)
                
                
                return iterations, queue, node

            if node.g < depth:
                queue.extend(
                  filter(
                    lambda child:
                      child.hash_titles() not in visited or
                      visited[child.hash_titles()] > child.g,
                    node.children()))
            
     
                #return
            if iterations%400000==0:
                print("count",iterations)
                 
with open('Input8PuzzleCasesUpdates.txt') as f:
    listRow = f.readlines();
    average_time=0
    Average_step=0
    Average_Iterations=0
    print("loading")
    for i in range(100):  
        start_time=time.clock()
        #string to list
        

        list_input=Node(np.array(listRow[i].rstrip().split(',')).astype(int).reshape((3,3)))
        
        count, queue, node =iterativeDeepeningDepthFirst(list_input)
        print("step:",i)
        print("success",node.data)
        a=len(node.parent)
        if i==0:
            for t in range(a):
                print_state.append(node.parent.pop(0))
        #print(node.data)
        endtime= time.clock()-start_time

        #print("time:", endtime)
        Average_Iterations=count+Average_Iterations
        Average_step=node.g+Average_step
        average_time=average_time+endtime
        print("================================================")
        #print("av_step",node.g)
        gc.collect()
    a="IterativeDeepening: " + str(Average_step/100) +"   "+ str(average_time/100)+"   "+str(Average_Iterations/100)
    print("Average :",a)
    result.append(a)

loading
count 400000
count 800000
count 1200000
step: 0
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
count 1200000
step: 1
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
step: 2
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
step: 3
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 4
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 5
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
count 1200000
count 1600000
count 2000000
step: 6
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 7
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 8
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 9
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
step: 10
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 11
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 12
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 13
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
count 1200000
step: 14
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
step: 15
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
ste

count 400000
count 800000
step: 74
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
step: 75
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
step: 76
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 77
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 78
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
count 1200000
step: 79
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 80
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 81
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
step: 82
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
step: 83
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
step: 84
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
step: 85
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
step: 86
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
step: 87
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
step: 88
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
count 1200000
step: 89
success [[0 1 2]
 [3 4 5]
 [6 7 8]]
count 400000
count 800000
count

In [3]:
#implementation of function "aStarMisplacedTiles"
#only class
class State:
    def __init__(self,data):
        self.data=data
        self.g = 0
        self.h = 0
        self.check=1 #false , 0 is True
        self.branch=[None,None,None,None]# Up,Down,Left,Right
        self.f = 0
        
    def swap(self, index1, index2):
        
        temp = self.data[index1]
        self.data[index1] = self.data[index2]
        self.data[index2] = temp
        
        return self
        
    def h_misplaced(self):
        h=0
        k=0
        
        for i in range(len(self.data)):
                if self.data[i] != GOAL_STATE[k] and self.data[i] !=0:
                    h=h+1
                k=k+1
        if h==0:
            print("successful",self.data)
            self.check=0 #success
        
        self.h=h
        
       
            #f=g+h
        self.f=self.g + h
        
    def move(self):
        
        index = self.data.index(0)
        #right
        if index % 3 == 2: 
            self.branch[3] = None
        else:
            self.branch[3]=State(self.data)
            self.branch[3]=copy.deepcopy(self)
            #print(self.data)
            
            self.branch[3] = self.branch[3].swap(index,index+1 )
            #print("self.branch[3].g:",self.branch[3].g)
            self.branch[3].g =self.g + 1
            self.branch[3].h_misplaced()
        #left
        if index % 3 == 0: 
            self.branch[2] = None
        else:
            self.branch[2]=State(self.data)
            self.branch[2]=copy.deepcopy(self)
            
            self.branch[2] = self.branch[2].swap(index,index-1 )
            
            self.branch[2].g=self.g+1
            self.branch[2].h_misplaced()
        #down
        if index >5: 
            self.branch[1] = None
        else:
            self.branch[1]=State(self.data)
            self.branch[1]=copy.deepcopy(self)
            
            self.branch[1] = self.branch[1].swap(index,index+3 )
            
            self.branch[1].g=self.g+1
            self.branch[1].h_misplaced()
            
        #up
        if index <3: 
            self.branch[0] = None
        else:
            self.branch[0]=State(self.data)
            self.branch[0]=copy.deepcopy(self)
            
            self.branch[0] = self.branch[0].swap(index,index-3 )
            
            self.branch[0].g=self.g+1
            self.branch[0].h_misplaced()
        
        


In [4]:
#run aStarMis
def aStarMis_search(list_input):
    explored = set()
    puzzle = []
    puzzle_cp= set()
    puzzle.append(State(list_input))
    puzzle_cp.add(tuple(list_input))
    
    maxNodes = 0
    count = 0

    while(len(puzzle) != 0):
        
        state = puzzle.pop(0)
        puzzle_cp.remove(tuple(state.data))
      
        
        if state.check == 0:
            #print("step: g(n)=",state.g,)
            return state.data ,state.g,count
        
        explored.add(tuple(state.data))
        
        state.move()
        count+=1
        
        for x in range(4):
            if state.branch[x] != None:
                    #print("x",x,state.branch[x].data)
                    if tuple(state.branch[x].data) not in explored and tuple(state.branch[x].data) not in puzzle_cp:
                        
                        puzzle_cp.add(tuple(state.branch[x].data))
                        if len(puzzle) ==0:
                            puzzle.insert(0, state.branch[x])
                        elif state.branch[x].h < puzzle[0].h:
                            puzzle.insert(0, state.branch[x])
                        else:
                            puzzle.insert(1, state.branch[x])
        
        #index 0 Is min F
        
        #puzzle= sorted(puzzle, key=lambda state: state.h,reverse=False)
        
        
 
        
        #print("point1:",point1)
        if count%1000==0:
            print("expanding: g(n)=",state.g,"h(n)=",state.h)
            #print("count",count)   
            pass
            
        if maxNodes < len(puzzle):
            maxNodes = len(puzzle)
        

    print ('Failure!')
    
    

with open('Input8PuzzleCasesUpdates.txt') as f:
    listRow = f.readlines();
    average_time=0
    average_step=0
    average_Iterations=0
    for i in range(100):  
        start_time=time.clock()
        #string to list
        list_input = listRow[i].split(",")
        list_input = [int(x) for x in list_input]
        
        #list_input = [1,2,3,4,8,0,7,6,5]
        print("print list:",list_input)
        start_time=time.clock()
        state,step, count=aStarMis_search(list_input)
        #state=aStarMis_search([1,2,3,4,8,0,7,6,5])
        print("print list successful:",i,":",state)
        endtime= time.clock()-start_time
        print("time:", endtime)
        average_step=average_step+step
        average_Iterations=average_Iterations+count
        average_time=average_time+endtime
        print("================================================")
        
        #listinput = np.matrix(listRow[i])
        #a= listinput.reshape((3,3));
        #
        #print(a)
         
    
    
    a="aStarMisplacedTiles: " + str(Average_step/100) +"   "+ str(average_time/100)+"   "+str(average_Iterations/100)
    print("Average :",a)
    result.append(a)
    

print list: [8, 7, 5, 4, 1, 2, 3, 0, 6]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 0 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.3600959999994302
print list: [7, 8, 6, 0, 2, 5, 1, 4, 3]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 1 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.2685590000000957
print list: [1, 0, 7, 5, 3, 6, 8, 4, 2]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 2 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.2304809999995996
print list: [3, 8, 4, 0, 1, 7, 5, 2, 6]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 3 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.2055209999998624
print list: [6, 1, 2, 7, 0, 8, 5, 4, 3]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 4 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.4689310000003388
print list: [2, 5, 3, 1, 4, 6, 8, 0, 7]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 5 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.23269600000003265
print list: [6, 8, 5, 3, 4, 7, 2, 1, 0]
succe

successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 38 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.6817240000000311
print list: [0, 5, 4, 2, 6, 1, 8, 3, 7]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 39 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.931152000000111
print list: [8, 4, 6, 3, 2, 7, 5, 0, 1]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 40 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.7069639999999708
print list: [3, 6, 8, 4, 2, 7, 1, 5, 0]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 41 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.6126150000000052
print list: [0, 6, 2, 7, 1, 5, 3, 4, 8]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 42 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.3049250000003667
print list: [3, 5, 6, 0, 1, 4, 2, 8, 7]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 43 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.25304400000004534
print list: [4, 0, 7, 3, 6, 2, 1, 5, 8]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
p

successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 77 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.5450040000005174
print list: [1, 2, 3, 5, 4, 0, 6, 8, 7]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 78 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.17480499999965105
print list: [2, 0, 6, 7, 8, 4, 5, 3, 1]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 79 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.828839999999218
print list: [7, 2, 8, 4, 6, 0, 3, 1, 5]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 80 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.5267540000004374
print list: [7, 0, 6, 4, 1, 2, 8, 3, 5]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 81 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.9670620000006238
print list: [3, 5, 4, 7, 0, 6, 1, 2, 8]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
print list successful: 82 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.7071370000003299
print list: [8, 2, 7, 0, 6, 5, 4, 1, 3]
successful [0, 1, 2, 3, 4, 5, 6, 7, 8]
p

In [5]:
#implementation of function "aStarManhattanDistance"
GOALIND = [(0, (0, 0)),(8, (2, 2)),  (1, (0, 1)), (2, (0, 2)), (3, (1, 0)), (4, (1, 1)), (6, (1, 2)), (6, (2, 0)), (7, (2, 1))]

class Manhattan(State):
    def h_manhattan(self):
        h=0
        k=0
        self.data
        
        for i in range(len(self.data)):
            if self.data[i] == 0:
                k=k+1
                continue
            if self.data[i] != GOAL_STATE[k]:
                #行
                t=int(i/3)            
                #列
                j=i%3
                num = self.data[i]
                x = GOALIND[num][1][0]
                y = GOALIND[num][1][1]
                distance = abs(x - t) + abs(y - j)
                h=h+distance
                
            k=k+1
        if h==0:
            self.check=0
        self.h=h
        self.f= self.g +h
        
    def move(self):
        
        index = self.data.index(0)
        #right
        if index % 3 == 2: 
            self.branch[3] = None
        else:
            self.branch[3]=State(self.data)
            self.branch[3]=copy.deepcopy(self)
            #print(self.data)
            
            self.branch[3] = self.branch[3].swap(index,index+1 )
            #print("self.branch[3].g:",self.branch[3].g)
            self.branch[3].g =self.g + 1
            self.branch[3].h_manhattan()
        #left
        if index % 3 == 0: 
            self.branch[2] = None
        else:
            self.branch[2]=State(self.data)
            self.branch[2]=copy.deepcopy(self)
            
            self.branch[2] = self.branch[2].swap(index,index-1 )
            
            self.branch[2].g=self.g+1
            self.branch[2].h_manhattan()
        #down
        if index >5: 
            self.branch[1] = None
        else:
            self.branch[1]=State(self.data)
            self.branch[1]=copy.deepcopy(self)
            
            self.branch[1] = self.branch[1].swap(index,index+3 )
            
            self.branch[1].g=self.g+1
            self.branch[1].h_manhattan()
            
        #up
        if index <3: 
            self.branch[0] = None
        else:
            self.branch[0]=State(self.data)
            self.branch[0]=copy.deepcopy(self)
            
            self.branch[0] = self.branch[0].swap(index,index-3 )
            
            self.branch[0].g=self.g+1
            self.branch[0].h_manhattan()
        
def aStarMan_search(list_input):
    explored = set()
    puzzle = []
    puzzle_cp=set()
    puzzle_cp.add(tuple(list_input))
    puzzle.append(Manhattan(list_input))
    
    maxNodes = 0
    count = 0

    while(len(puzzle) != 0):
        totle_time=time.clock
        
        state = puzzle.pop(0)
        puzzle_cp.remove(tuple(state.data))
        #print("current state f",state.f)
        #print("old",explored)
        
        #if tuple (state.data) in  explored :
        #   print("move",state.data)
        #    state = puzzle.pop(0)
        #print("new",explored)
        
        if state.check == 0:
            
            del explored
            del puzzle
            del puzzle_cp
            #print("expanding: g(n)=",state.g,)
            #print("Max node",maxNodes)
            return state.data,state.g,count
        
        
        state.move()
        
        
        count+=1
        explored.add(tuple(state.data))
        for x in range(4):
            if state.branch[x] != None:
                    
                    if tuple(state.branch[x].data) not in explored and tuple(state.branch[x].data) not in puzzle_cp:
                        
                        puzzle_cp.add(tuple(state.branch[x].data))
                        if len(puzzle) ==0:
                            puzzle.insert(0, state.branch[x])
                        elif state.branch[x].h < puzzle[0].h:
                            puzzle.insert(0, state.branch[x])
                        else:
                            puzzle.insert(1, state.branch[x])
        
        #算F（n）平均time是6.9， 只算h为4.5
        
        #puzzle= sorted(puzzle, key=lambda state: state.h,reverse=False)
       
        #print("data:",puzzle[0].data)
        #print("g:",state.g)
        #print("h:",state.h)
       
        
        if count%1000==0:
            print("expanding: g(n)=",state.g,"h(n)=",state.h)
            print("count",count)   
        if maxNodes < len(puzzle):
            maxNodes = len(puzzle)

        
        
    print ('Failure!')
    
    

with open('Input8PuzzleCasesUpdates.txt') as f:
    listRow = f.readlines();
    average_time=0
    
    average_step=0
    average_Iterations=0
    for i in range(100):  
        start_time=time.clock()
        #string to list
        list_input = listRow[i].split(",")
        list_input = [int(x) for x in list_input]
        
        
        print("print list:",list_input)
        state,step,count =aStarMan_search(list_input)
        
        print("print list successful:",i,":",state)
        endtime= time.clock()-start_time
        print("time:", endtime)
        average_step=average_step+step
        average_Iterations=average_Iterations+count
        average_time=average_time+endtime
        print("================================================")
        #listinput = np.matrix(listRow[i])
        #a= listinput.reshape((3,3));
        #
        #print(a)
        
    
    print("Average time:",average_time/10)
    a="aStarManhattanDistance: " + str(Average_step/100) +"   "+ str(average_time/100)+"   "+str(average_Iterations/100)
    result.append(a)

print list: [8, 7, 5, 4, 1, 2, 3, 0, 6]
print list successful: 0 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.4718459999994593
print list: [7, 8, 6, 0, 2, 5, 1, 4, 3]
print list successful: 1 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.36805699999968056
print list: [1, 0, 7, 5, 3, 6, 8, 4, 2]
print list successful: 2 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.587336999999934
print list: [3, 8, 4, 0, 1, 7, 5, 2, 6]
print list successful: 3 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.586913000000095
print list: [6, 1, 2, 7, 0, 8, 5, 4, 3]
print list successful: 4 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.6465879999996105
print list: [2, 5, 3, 1, 4, 6, 8, 0, 7]
print list successful: 5 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.2557150000002366
print list: [6, 8, 5, 3, 4, 7, 2, 1, 0]
print list successful: 6 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.5885570000000371
print list: [2, 7, 8, 0, 1, 5, 4, 3, 6]
print list successful: 7 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.3522060000004785
print list: [6, 5, 2, 1, 4, 8, 0, 7, 3]
p

print list successful: 47 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.5514100000000326
print list: [2, 0, 5, 4, 7, 3, 6, 8, 1]
print list successful: 48 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.0916800000004514
print list: [7, 2, 0, 4, 1, 6, 5, 3, 8]
print list successful: 49 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.19396800000049552
print list: [6, 4, 8, 1, 3, 7, 2, 5, 0]
expanding: g(n)= 695 h(n)= 6
count 1000
expanding: g(n)= 1361 h(n)= 4
count 2000
expanding: g(n)= 2039 h(n)= 6
count 3000
print list successful: 50 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 3.591131000000132
print list: [8, 3, 6, 5, 0, 2, 1, 7, 4]
print list successful: 51 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.5703599999997095
print list: [6, 2, 5, 8, 1, 3, 7, 4, 0]
print list successful: 52 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.7127539999992223
print list: [4, 3, 2, 0, 5, 8, 1, 6, 7]
print list successful: 53 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.11338300000079471
print list: [5, 1, 4, 0, 3, 7, 2, 8, 6]
print list successful: 54 : [0

print list successful: 95 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.6010120000000825
print list: [1, 2, 4, 5, 3, 6, 7, 0, 8]
print list successful: 96 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.6425710000003164
print list: [6, 5, 1, 0, 8, 3, 4, 7, 2]
print list successful: 97 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.15455300000030547
print list: [7, 5, 8, 3, 4, 1, 0, 6, 2]
print list successful: 98 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.15810999999939668
print list: [6, 5, 7, 3, 8, 0, 4, 1, 2]
print list successful: 99 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.4478669999998601
Average time: 5.143966499999533


In [6]:
#implementation of function "print_result(result)"
def print_result(result):
    print("Solution of the first Scenario")
    for pri in print_state:
        print(pri)
        print('\n')
        print("to")
    
    s = "Name          Average_Steps    Average_Time   Average_Iterations"+'\n' 
    if result == None:
        return
    else:
        
        for x in range(len(result)):
            s = s+result[x]+' '
            s = s + '\n'
        print(s)
    
#print("Name    Average_Steps    Average_Time   Average_Iterations ")

In [7]:
#implementation of function "breadthFirstSearch" (for graduate students)


You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
===============================

In [8]:
#you need to implement print_result function to print out the result according to the required format
print_result(result)

Solution of the first Scenario
[[8 7 5]
 [4 1 2]
 [3 0 6]]


to
[[8 7 5]
 [4 0 2]
 [3 1 6]]


to
[[8 7 5]
 [4 2 0]
 [3 1 6]]


to
[[8 7 0]
 [4 2 5]
 [3 1 6]]


to
[[8 0 7]
 [4 2 5]
 [3 1 6]]


to
[[8 2 7]
 [4 0 5]
 [3 1 6]]


to
[[8 2 7]
 [4 1 5]
 [3 0 6]]


to
[[8 2 7]
 [4 1 5]
 [3 6 0]]


to
[[8 2 7]
 [4 1 0]
 [3 6 5]]


to
[[8 2 0]
 [4 1 7]
 [3 6 5]]


to
[[8 0 2]
 [4 1 7]
 [3 6 5]]


to
[[0 8 2]
 [4 1 7]
 [3 6 5]]


to
[[4 8 2]
 [0 1 7]
 [3 6 5]]


to
[[4 8 2]
 [1 0 7]
 [3 6 5]]


to
[[4 0 2]
 [1 8 7]
 [3 6 5]]


to
[[0 4 2]
 [1 8 7]
 [3 6 5]]


to
[[1 4 2]
 [0 8 7]
 [3 6 5]]


to
[[1 4 2]
 [3 8 7]
 [0 6 5]]


to
[[1 4 2]
 [3 8 7]
 [6 0 5]]


to
[[1 4 2]
 [3 0 7]
 [6 8 5]]


to
[[1 4 2]
 [3 7 0]
 [6 8 5]]


to
[[1 4 2]
 [3 7 5]
 [6 8 0]]


to
[[1 4 2]
 [3 7 5]
 [6 0 8]]


to
[[1 4 2]
 [3 0 5]
 [6 7 8]]


to
[[1 0 2]
 [3 4 5]
 [6 7 8]]


to
[[0 1 2]
 [3 4 5]
 [6 7 8]]


to
Name          Average_Steps    Average_Time   Average_Iterations
IterativeDeepening: 22.33   52.53634803000002 


The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Solution of the first Scenario:
X X X
X X X
X X X
to
X X X
X X X
X X X
to
X X X
X X X
X X X
to
X X X
X X X
X X X
to
.
.
.
0 1 2
3 4 5
6 7 8

                Average_Steps    Average_Time   Average_Iterations   
UCS
A*(Misplaced)
A*(Manhattan)
BFS

